In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os

import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.metrics import adjusted_rand_score as ARI
from sklearn.metrics import adjusted_mutual_info_score as AMI

from scipy import sparse
from matplotlib import rcParams
rcParams['pdf.fonttype'] = 42 # enables correct plotting of text
rcParams['figure.figsize'] = (12,12)
import seaborn as sns
from scipy import sparse

sc.settings.verbosity =0

In [5]:
mod ='scRNA'

In [6]:
adata = sc.read('../MODALITY_INTEGRATION/objects/Integration_2ndQC_metadata.h5ad', compression='gzip')

In [7]:
samples_to_filter = ['lib_776','lib_23']#,'lib_03','lib_420','lib_49','lib_68']#,'lib_36','lib_38','lib_34']

In [8]:
adata = adata[~adata.obs['sample'].isin(samples_to_filter)].copy()

In [9]:
adata.X = adata.layers['counts'].copy()

In [10]:
adata.obs['batch'].value_counts()

snRNA      69036
scRNA      65705
scRNA5p    11265
Name: batch, dtype: int64

In [11]:
adata = adata[adata.obs['batch'].isin(['scRNA'])].copy()

In [12]:
adata.obs

,sample,batch,n_counts,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,total_counts_rb,pct_counts_rb,Deepscore_external,Doublet_Detection_doublets_scRNA,Doublet_Detection_doublets_score_scRNA,Doublet_Detection_doublets_scRNA5p,Doublet_Detection_doublets_score_scRNA5p,AMULET_doublets,AMULET_pval,AMULET_qval,Doublet_Detection_doublets_snRNA,Doublet_Detection_doublets_score_snRNA
AAAGAACAGGTAAAGG-1-0-scRNA,lib_55,scRNA,1583.0,785,1583.0,661.0,41.756161,72.0,4.548326,Unclassified,1.0,57.652243,NaN,NaN,False,NaN,NaN,NaN,NaN
AAAGTCCCAATCGAAA-1-0-scRNA,lib_55,scRNA,6136.0,2394,6136.0,1248.0,20.338982,872.0,14.211212,Unclassified,0.0,38.928254,NaN,NaN,False,NaN,NaN,NaN,NaN
AACAAGAAGACCTTTG-1-0-scRNA,lib_55,scRNA,2563.0,792,2563.0,1532.0,59.773701,146.0,5.696449,Unclassified,0.0,38.444448,NaN,NaN,False,NaN,NaN,NaN,NaN
AACCAACGTATCCCTC-1-0-scRNA,lib_55,scRNA,1546.0,477,1546.0,889.0,57.503235,120.0,7.761966,Unclassified,0.0,28.275289,NaN,NaN,False,NaN,NaN,NaN,NaN
AAGACTCAGGTAAACT-1-0-scRNA,lib_55,scRNA,1222.0,310,1222.0,767.0,62.765957,138.0,11.292962,Unclassified,0.0,28.275289,NaN,NaN,False,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGCCTGTCTAGCATG-1-19-scRNA,lib_49,scRNA,1874.0,736,1874.0,681.0,36.339382,235.0,12.540022,PT,0.0,0.024093,NaN,NaN,False,NaN,NaN,NaN,NaN
TTGTTGTGTTTCACTT-1-19-scRNA,lib_49,scRNA,5209.0,2163,5209.0,1274.0,24.457668,133.0,2.553273,PT,0.0,1.626684,NaN,NaN,False,NaN,NaN,NaN,NaN
TTGTTGTTCGAAGCCC-1-19-scRNA,lib_49,scRNA,1127.0,633,1127.0,379.0,33.629105,70.0,6.211180,PT,0.0,0.024093,NaN,NaN,False,NaN,NaN,NaN,NaN
TTTACTGTCAACGCTA-1-19-scRNA,lib_49,scRNA,2603.0,1191,2603.0,502.0,19.285440,253.0,9.719554,PT,0.0,0.024093,NaN,NaN,False,NaN,NaN,NaN,NaN


In [13]:
obs = pd.read_csv('../MODALITY_INTEGRATION/Multi_obs.csv', index_col=0)

/tmp/ipykernel_50364/3819850974.py:1: DtypeWarning: Columns (4,5,6,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  obs = pd.read_csv('../MODALITY_INTEGRATION/Multi_obs.csv', index_col=0)


In [14]:
obs = obs[obs['batch'].isin(['scRNA'])]
obs.index = [i.split('_')[0] for i in obs.index]

In [15]:
adata.obs['In_final_obj'] = [i in obs.index for i in adata.obs.index]
adata.obs['In_final_obj'].value_counts()

True     43502
False    22203
Name: In_final_obj, dtype: int64

In [16]:
adata.obs['In_final_obj'] = adata.obs['In_final_obj'].astype('category')

In [18]:
adata = adata[adata.obs['In_final_obj'].astype(bool)].copy()

In [19]:
adata.obs['sample'].value_counts()

lib_19     5574
lib_68     4218
lib_38     3627
lib_51     3513
lib_49     3246
lib_420    2743
lib_15     2426
lib_56     2323
lib_16     2126
lib_03     1968
lib_57     1931
lib_09     1813
lib_29     1792
lib_55     1689
lib_20     1673
lib_10     1578
lib_36     1002
lib_34      260
Name: sample, dtype: int64

In [ ]:
## Filter Out cell with less than 500 genes - In single cell 3' less than 500 genes is a clear indicative of low quality - We dont want this confounding factor in our analysis

In [21]:
filter_mask = (adata.obs['n_genes_by_counts'] < 500 & ~adata.obs['Deepscore_external'].isin(['LEUK']))
np.sum(filter_mask)

7989

In [22]:
adata = adata[~filter_mask].copy()
adata

AnnData object with n_obs × n_vars = 35513 × 38224
    obs: 'sample', 'batch', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_rb', 'pct_counts_rb', 'Deepscore_external', 'Doublet_Detection_doublets_scRNA', 'Doublet_Detection_doublets_score_scRNA', 'Doublet_Detection_doublets_scRNA5p', 'Doublet_Detection_doublets_score_scRNA5p', 'AMULET_doublets', 'AMULET_pval', 'AMULET_qval', 'Doublet_Detection_doublets_snRNA', 'Doublet_Detection_doublets_score_snRNA', 'In_final_obj', 'Deepscore'
    uns: 'batch_colors', 'log1p', 'neighbors', 'pca', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

In [23]:
sc.pp.highly_variable_genes(adata, n_top_genes=2000, flavor = 'seurat_v3', n_bins=1)

In [24]:
adata.write('objects/scRNA_raw.h5ad', compression='gzip')